**TABU SEARCH**
---

---

# INTRODUCCIÓN

- Una **metaheurística** es una técnica diseñada para resolver problemas con una solución aproximada donde los métodos clásicos fallan en encontrar una solución exacta en una cantidad de tiempo razonable (problema NP-hard).
- El **Single Machine Total Weighted Tardiness Problem (SMTWTP)** es un problema NP-hard donde se tiene una máquina que puede manejar un trabajo a la vez, y hay $N$ trabajos a ser procesados sin la interrupciones en la máquina.
- Cada trabajo $i \in N$ requiere un tiempo de procesamiento $P_i$, y tiene un peso positivo $W_i$ que indica la importancia del trabajo y una fecha de vencimiento $d_i$.
- Si se asume que la máquina está disponible para el procesamiento en el tiempo cero, se puede indicar el tiempo de finalización del trabajo $i$ como $C_i$, y la tardanza del trabajo se puede calcular como $T_i = max(C_i - d_i, 0)$, por lo que si el trabajo es procesado antes de su fecha de vencimiento $C_i \leq d_i$, no habrá tardanza $T = 0$.
- El objetivo es ordenar los $N$ trabajos de tal manera que se minimice la tardanza ponderada total de todo el proceso, es decir $min \sum W_i T_i$, donde si un trabajo tiene un peso mayor, la penalización por tardanza será mayor.

# ALGORITMO DE TABU SEARCH (TS)

El algoritmo TS fue propuesto por Glover en 1986 y desarrollado también en paralelo por Hansen, y se aplica a problemas de optimización intentando encontrar la mejor solución admisible en el vecindario de la solución actual en cada iteración, considerando las soluciones recientes como un "tabú", para evitar los ciclos.

**PASOS DEL ALGORITMO**

**Paso 0:** Se crea una solución inicial para que el algoritmo pueda iterar sobre ésta y encontrar una mejor. La solución inicial se puede ver como un punto de partida del algoritmo, y en la mayoría de los casos, se asigna de manera aleatoria. Sin embargo, si se tiene un buen entendimiento del problema, es posible diseñar un algoritmo específico para construir la solución inicial.

**Paso 1:** Ahora que se tiene la solución inicial, el siguiente paso es crear la lista de soluciones candidatas a partir de la solución actual $S$ (solución inicial en la iteración 0), a las que llamamos **soluciones vecinas** o **vecindad de $S$**. Para encontrar las soluciones vecinas de la solución actual $S$, se necesita definir una función de vecindario, bajo la cual cada solución $S$ tiene asociado un subconjunto de soluciones. A manera de ejemplo asumiremos que la solución actual en el SMTWTP es $S = [3, 8, 10, 4, 1, 6, 2, 5, 9, 7]$, y definimos nuestra función de vecindario como un movimiento de intercambio $N(S, Swap)$, es decir, reemplazar el orden de dos trabajos, por lo tanto, una solución de vecindario sería $[8, 3, 10, 4, 1, 6, 2, 5, 9, 7]$, donde los trabajos $8$ y $3$ son intercambiados. Otra solución sería $[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]$, intercambiando los trabajos $5$ y $10$. En conclusión, el número de soluciones de vecindad de $S$ realizando el movimiento de intercambio es $(n-2)$ para cada uno de los $n$ trabajos, lo que en una notación Big-O sería $O(n2)$.

```
Initial Random Solution: [3, 8, 10, 4, 1, 6, 2, 5, 9, 7]
Neighbor Solution: [3, 8, 5, 4, 1, 6, 2, 10, 9, 7]
```

**Paso 2:** A partir de las soluciones de vecindario creadas en el paso 1, se elige la mejor solución admisible (no tabú o que cumple con los criterios de aspiración), marcando cada solución como en el diagrama de a continuación:

![tabu](tabu.png)

> Antes de continuar al paso 3, se definirán dos conteptos importantes para el algoritmo: **Lista tabú** y **Criterios de aspiración**.

**LISTA TABÚ**

Es la lista que el algoritmo TS utiliza para registrar las soluciones recientes y de esta manera evitar que se vuelva a entrar en estas soluciones después de un número específico de iteraciones, lo que ayuda a que la búsqueda se aleje de las soluciones que ya se visitaron, y por lo tanto, se realiza una exploración más extensa.

**Tabu Tenure:** Define el tamaño de la lista tabú, es decir, define por cuántas iteraciones una solución se mantiene como tabú. En este ejemplo, la tabu tenure es $2$. En $iteration - 1$ la lista tabú está vacía, es decir lista tabu = {}. Asumiremos que la mejor solución admisible encontrada en $iteration - 1$ es $[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]$ (de acuerdo a los pasos 1 y 2), por lo tanto la lista tabú se convierte a ${[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]}$, en $iteration-2$, la mejor solución sigue siendo $[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]$, sin embargo, no es admisible porque está en tabú, por lo que tomaremos la siguiente mejor solución, la cual asumiremos que es $[8, 3, 5, 4, 1, 6, 2, 10, 7,9]$, ahora la lista tabú es ${[8, 3, 5, 4, 1, 6, 2, 10, 9, 7], [8, 3, 5, 4, 1, 6, 2, 10, 7,9]}$. En $iteration-3$, la solución $[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]$ deja la lista tabú, y la mejor solución admisible entra en la lista (porque el tamaño de la lista es 2), y así sucesivamente. Hay que tener en cuenta que la tabu tenure tiene un gran impacto en el rendimiento de la TS ya que entre menor sea la tabu tenure, hay una mayor probabilidad de caer en un ciclo. En términos generales, no existe una regla simple para establecer la tabu tenure, ya que depende en gran medida del problema que se está tratando de resolver.

**Atritbuto Tabú:** Si se tiene una tabu tenure grande con problemas grandes, registrar las soluciones en la lista tabú será muy costoso computacionalmente. Para solucionar esto, se suelen alamacenar únicamente los movimientos realizados en lugar de toda la solución. Los atributos tabú definen el componente de la solución que se mantiene en la lista tabú. Para el ejemplo descrito anteriormente en la tabu tenure, podríamos almacenar los trabajos que han sido intercambiados como lista tabu $= {(8,3), (7,9)}$, lo que significa, que no se deberán intercambiar los trabajos $(8,3)$ o $(7,9)$ por las siguientes dos iteraciones.

**CRITERIOS DE ASPIRACIÓN**

Si la solución encontrada en la iteración actual tiene un valor (de la función objetivo) que se mejor que el valor de la mejor solución conocida actualmente, pero el movimiento es tabú, podemos usar el criterio de aspiración para anular el estado tabú de éste movimiento, incluyendo así la solución excluida en el conjunto permitido.

> Volvemos con la descripción de los pasos del algoritmo.

**Paso 3:** Revisar los criterios de detención definidos, que pueden ser un número de iteraciones alcanzadas o un tiempo de ejecución. Si no se cumplen los criterios de detención, se continua en el paso 4, o en caso contrario, el algoritmo termina y se devuelve la mejor solución encontrada.

**Paso 4:** Se actualiza la lista tabú y los criterios de aspiración y se regresa al paso 1.

> **Nota:** El algoritmo TS que se explica en el ejemplo involucra el manejo de memoria a corto plazo, que en ocaciones puede resolver satisfactoriamente problemas complejos, pero en la mayoría de los casos, se necesitarán elementos adicionales en la estrategia de búsqueda, tales como la intensificación y diversificación para encontrar la mejor solución posible.

# IMPLEMENTACIÓN

In [1]:
import pandas as pd
import random as rd
from itertools import combinations
import math

class TS():
    def __init__(self, Path, seed, tabu_tenure):
        self.Path = Path
        self.seed = seed
        self.tabu_tenure = tabu_tenure
        self.instance_dict = self.input_data()
        self.Initial_solution = self.get_InitialSolution()
        self.tabu_str, self.Best_solution, self.Best_objvalue = self.TSearch()


    def input_data(self):
        '''Takes the path of the excel file of the SMTWTP instances.
        Returns a dict of jobs number as Key and weight, processing time (hours) and due date (hours) as values.
        '''
        return pd.read_csv(self.Path, index_col=0).to_dict('index')

    def get_tabuestructure(self):
        '''Takes a dict (input data)
        Returns a dict of tabu attributes(pair of jobs that are swapped) as keys and [tabu_time, MoveValue]
        '''
        dict = {}
        for swap in combinations(self.instance_dict.keys(), 2):
            dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
        return dict

    def get_InitialSolution(self, show=False):
        n_jobs = len(self.instance_dict) # Number of jobs
        # Producing a random schedule of jobs
        initial_solution = list(range(1, n_jobs+1))
        rd.seed(self.seed)
        rd.shuffle(initial_solution)
        if show == True:
            print("initial Random Solution: {}".format(initial_solution))
        return initial_solution

    def Objfun(self, solution, show = False):
        '''Takes a set of scheduled jobs, dict (input data)
        Return the objective function value of the solution
        '''
        dict = self.instance_dict
        t = 0   #starting time
        objfun_value = 0
        for job in solution:
            C_i = t + dict[job]["processing_time"]  # Completion time
            d_i = dict[job]["due_date"]   # due date of the job
            T_i = max(0, C_i - d_i)    #tardiness for the job
            W_i = dict[job]["weight"]  # job's weight

            objfun_value +=  W_i * T_i
            t = C_i
        if show == True:
            print("\n","#"*8, "The Objective function value for {} solution schedule is: {}".format(solution ,objfun_value),"#"*8)
        return objfun_value

    def SwapMove(self, solution, i ,j):
        '''Takes a list (solution)
        returns a new neighbor solution with i, j swapped
       '''
        solution = solution.copy()
        # job index in the solution:
        i_index = solution.index(i)
        j_index = solution.index(j)
        #Swap
        solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
        return solution

    def TSearch(self):
        '''The implementation Tabu search algorithm with short-term memory and pair_swap as Tabu attribute.
        '''
        # Parameters:
        tenure =self.tabu_tenure
        tabu_structure = self.get_tabuestructure()  # Initialize the data structures
        best_solution = self.Initial_solution
        best_objvalue = self.Objfun(best_solution)
        current_solution = self.Initial_solution
        current_objvalue = self.Objfun(current_solution)

        print("#"*30, "Short-term memory TS with Tabu Tenure: {}\nInitial Solution: {}, Initial Objvalue: {}".format(
            tenure, current_solution, current_objvalue), "#"*30, sep='\n\n')
        iter = 1
        Terminate = 0
        while Terminate < 100:
            print('\n\n### iter {}###  Current_Objvalue: {}, Best_Objvalue: {}'.format(iter, current_objvalue,
                                                                                    best_objvalue))
            # Searching the whole neighborhood of the current solution:
            for move in tabu_structure:
                candidate_solution = self.SwapMove(current_solution, move[0], move[1])
                candidate_objvalue = self.Objfun(candidate_solution)
                tabu_structure[move]['MoveValue'] = candidate_objvalue

            # Admissible move
            while True:
                # select the move with the lowest ObjValue in the neighborhood (minimization)
                best_move = min(tabu_structure, key =lambda x: tabu_structure[x]['MoveValue'])
                MoveValue = tabu_structure[best_move]["MoveValue"]
                tabu_time = tabu_structure[best_move]["tabu_time"]
                # Not Tabu
                if tabu_time < iter:
                    # make the move
                    current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = self.Objfun(current_solution)
                    # Best Improving move
                    if MoveValue < best_objvalue:
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Best Improving => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                    else:
                        print("   ##Termination: {}## best_move: {}, Objvalue: {} => Least non-improving => "
                              "Admissible".format(Terminate,best_move,
                                                                                                           current_objvalue))
                        Terminate += 1
                    # update tabu_time for the move
                    tabu_structure[best_move]['tabu_time'] = iter + tenure
                    iter += 1
                    break
                # If tabu
                else:
                    # Aspiration
                    if MoveValue < best_objvalue:
                        # make the move
                        current_solution = self.SwapMove(current_solution, best_move[0], best_move[1])
                        current_objvalue = self.Objfun(current_solution)
                        best_solution = current_solution
                        best_objvalue = current_objvalue
                        print("   best_move: {}, Objvalue: {} => Aspiration => Admissible".format(best_move,
                                                                                                      current_objvalue))
                        Terminate = 0
                        iter += 1
                        break
                    else:
                        tabu_structure[best_move]["MoveValue"] = float('inf')
                        print("   best_move: {}, Objvalue: {} => Tabu => Inadmissible".format(best_move,
                                                                                              current_objvalue))
                        continue
        print('#'*50 , "Performed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,best_objvalue), sep="\n")
        return tabu_structure, best_solution, best_objvalue


test = TS(Path="Instance_10.csv", seed = 2012, tabu_tenure=3)

##############################

Short-term memory TS with Tabu Tenure: 3
Initial Solution: [4, 7, 9, 1, 5, 3, 10, 6, 8, 2], Initial Objvalue: 39.080000000000005

##############################


### iter 1###  Current_Objvalue: 39.080000000000005, Best_Objvalue: 39.080000000000005
   best_move: (2, 7), Objvalue: 26.13 => Best Improving => Admissible


### iter 2###  Current_Objvalue: 26.13, Best_Objvalue: 26.13
   best_move: (8, 9), Objvalue: 17.930000000000003 => Best Improving => Admissible


### iter 3###  Current_Objvalue: 17.930000000000003, Best_Objvalue: 17.930000000000003
   best_move: (3, 4), Objvalue: 14.960000000000003 => Best Improving => Admissible


### iter 4###  Current_Objvalue: 14.960000000000003, Best_Objvalue: 14.960000000000003
   best_move: (4, 5), Objvalue: 14.200000000000003 => Best Improving => Admissible


### iter 5###  Current_Objvalue: 14.200000000000003, Best_Objvalue: 14.200000000000003
   best_move: (6, 7), Objvalue: 13.830000000000002 => Best Improving 